<a href="https://colab.research.google.com/github/Hybern8/EntLife/blob/version3/Technical_dept_transfomer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Data Transformation process for extracting & transforming ET's transctions data into a more analytical format.

Process


1.   Download data from core application and save into designated folder
2.   Rename file to "Raw"
3.   Run this script by [Runtime > Run all]



### Load R magic on Python

In [ ]:
%load_ext rpy2.ipython

### Install "openxlsx", load other relevant libraries

In [ ]:
%%R

install.packages("openxlsx")

# Load in the libraries

library(dplyr) # for data manipulation
library(tidyr) # for data manipulation
library(openxlsx) # for data reading & writing

### Mount Google drive to access designated folders for data transformation and for the transformed data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### Designate folders

In [ ]:
folder_path = '/content/drive/Othercomputers/Decipher/Documents/Quarterhill/TechRep'

### Set working directory and confirm working directory is live

In [ ]:
%%R -i folder_path
setwd(folder_path)

# Verify the working directory
getwd()

### Read in raw data for renaming columns. Read in Raw data, split into usable columns and transform

In [ ]:
%%R
# Read in data for column names
col_names <- read.xlsx("Ref_file/col_names.xlsx", colNames = T)

# Read in data & split columns
Raw <- read.csv('dataset/Raw.csv',stringsAsFactors = F)
data <- Raw %>%
  separate(`ELAC.SEARCH.PAYER.EXTRACT.INDEX.1`,
           into = c("A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L"),
           sep = "~", fill = "right") %>%
  separate(`ELAC.SEARCH.PAYER.EXTRACT.INDEX.2`,
           into = c("A1", "B1", "C1", "D1", "E1", "F1", "G1", "H1", "I1", "J1", "K1", "L1"),
           sep = "~", fill = "right") %>%
  separate(`ELAC.SEARCH.PAYER.EXTRACT.INDEX.3`,
           into = c("A2", "B2", "C2", "D2", "E2", "F2", "G2", "H2", "I2", "J2"),
           sep = "~", fill = "right") %>%
  separate(`ELAC.SEARCH.PAYER.EXTRACT.INDEX.4`,
           into = c("A3", "B3", "C3", "D3", "E3", "F3", "G3", "H3", "I3"),
           sep = "~", fill = "right") %>%
  separate(`ELAC.SEARCH.PAYER.EXTRACT.INDEX.5`,
           into = c("A4", "B4", "C4", "D4", "E4", "F4"),
           sep = "~", fill = "right")

# Rename column names
colnames(data) <- colnames(col_names)

### Check data was read in by confirming data dimension

In [ ]:
%%R
dim(Raw)

### Move raw data into "used" folderto verify data was used for transformation

In [ ]:
%%R
# List of all the files in teh dataset folder
files <- list.files(path = "dataset", full.names = T)
# delete all the files to keep the folder ready for next job
# unlink(files) - Keep in view for v4
# Write used raw data into [Used] folder to keep track of jobs
write.csv(Raw, paste0("Used/Raw_", Sys.Date(), ".csv"), row.names = F)


### Create first & second dataframe, and select relevant folder, dropping useless ones

In [ ]:
%%R
# Fist report data selection
select_data <- data %>%
  select(`SURNAME-PAYER`,
         `FIRST-NAMES-PAYER`,
         `PAYER-PHONE-CELL`,
         `PAYER-PHONE-WORK`,
         `PAYER-PHONE-HOME`,
         `AGENT-ALIAS-NAME`,
         `CONTRACT-PAYMENT-STATUS-REASON`,
         `CONTRACT-PAYMENT-FREQUENCY`,
         VALUE2,
         `NUMBER-OF-OUTSTANDING-PAYMENTS`,
         `CONTRACT-ID`) %>%
  filter(`CONTRACT-PAYMENT-STATUS-REASON` %in% c("ACTIVE", "PAID-UP", "PAYMENT-LAPSED")) %>%
  filter(`NUMBER-OF-OUTSTANDING-PAYMENTS` > 1)

# Second report data selection
next_select_data <- data %>%
  select(`SURNAME-PAYER`,
         `FIRST-NAMES-PAYER`,
         `PAYER-PHONE-CELL`,
         `PAYER-PHONE-WORK`,
         `PAYER-PHONE-HOME`,
         `AGENT-ALIAS-NAME`,
         `CONTRACT-PAYMENT-STATUS-REASON`,
         `CONTRACT-PAYMENT-FREQUENCY`,
         VALUE2,
         `NUMBER-OF-OUTSTANDING-PAYMENTS`,
         `CONTRACT-ID`,
         `PREVIOUS-CONTRACT-PAYMENT-POSITION`) %>%
  filter(`CONTRACT-PAYMENT-STATUS-REASON` %in% c("ACTIVE", "PAID-UP", "PAYMENT-LAPSED")) %>%
  filter(`NUMBER-OF-OUTSTANDING-PAYMENTS` > 1) %>%
  filter(`PREVIOUS-CONTRACT-PAYMENT-POSITION` < 0)

### Write in and save output in designated folder

In [ ]:
%%R
# Write and save both files
cleaned_alpha <- write.xlsx(select_data, paste0("Cleaned/Cleaned_alpha_", Sys.Date(), ".xlsx"), row.names = F)
cleaned_beta <- write.xlsx(next_select_data, paste0("Cleaned/Cleaned_beta_", Sys.Date(), ".xlsx"), row.names = F)